In [1]:
!pip install nltk -U

     |████████████████████████████████| 1.5MB 3.7MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [33]:
from nltk.translate.bleu_score import *
from nltk.translate.meteor_score import *
import pandas as pd
import ast

In [3]:
import os
#os.system("git clone https://github.com/soerenetler/squad-question-generation.git")
#os.system("git clone https://github.com/soerenetler/spacy_target_answer.git")
#os.system("git clone https://github.com/soerenetler/crf_target_answer.git")

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [34]:
path_to_folder = "/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/squad/question_answer/"
df = pd.read_csv(path_to_folder+"dev.csv")
df["answer_sentence_token"] = [ast.literal_eval(t) for t in df["answer_sentence_token"]]
df["question_token"] = [ast.literal_eval(t) for t in df["question_token"]]

In [39]:
import unicodedata
def unicode_to_ascii(s):
    s = s.lower().strip()
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')
    
df["question_token"] = [[unicode_to_ascii(t)for t in question] for question in df["question_token"]]
df["answer_sentence_token"] = [[unicode_to_ascii(t)for t in question] for question in df["answer_sentence_token"]]

In [40]:
df.head()

,Unnamed: 0,text_title,question_text,question_token,question_pos,question_tag,question_parse_tree,answer_sentence,answer_sentence_token,answer_paragraph,masked_answer_paragraph,masked_answer_sentence,answer,unanswerable,sentence_question_bleu
0,2068,The_Legend_of_Zelda:_Twilight_Princess,What category of game is Legend of Zelda: Twil...,"[what, category, of, game, is, legend, of, zel...","['DET', 'NOUN', 'ADP', 'NOUN', 'AUX', 'PROPN',...","['WDT', 'NN', 'IN', 'NN', 'VBZ', 'NNP', 'IN', ...",NaN,The Legend of Zelda: Twilight Princess (Japane...,"[the, legend, of, zelda, :, twilight, princess...",The Legend of Zelda: Twilight Princess (Japane...,"['the', 'legend', 'of', 'zelda', ':', 'twiligh...","['the', 'legend', 'of', 'zelda', ':', 'twiligh...",action-adventure,False,1.017037e-232
1,2069,The_Legend_of_Zelda:_Twilight_Princess,What consoles can be used to play Twilight Pri...,"[what, consoles, can, be, used, to, play, twil...","['PRON', 'NOUN', 'AUX', 'AUX', 'VERB', 'PART',...","['WP', 'NNS', 'MD', 'VB', 'VBN', 'TO', 'VB', '...",NaN,The Legend of Zelda: Twilight Princess (Japane...,"[the, legend, of, zelda, :, twilight, princess...",The Legend of Zelda: Twilight Princess (Japane...,"['the', 'legend', 'of', 'zelda', ':', 'twiligh...","['the', 'legend', 'of', 'zelda', ':', 'twiligh...",GameCube and Wii,False,4.065974e-233
2,2070,The_Legend_of_Zelda:_Twilight_Princess,When was Twilight Princess launched in North A...,"[when, was, twilight, princess, launched, in, ...","['ADV', 'AUX', 'PROPN', 'PROPN', 'VERB', 'ADP'...","['WRB', 'VBD', 'NNP', 'NNP', 'VBN', 'IN', 'NNP...",NaN,The Wii version was released alongside the con...,"[the, wii, version, was, released, alongside, ...",The Legend of Zelda: Twilight Princess (Japane...,"['the', 'legend', 'of', 'zelda', ':', 'twiligh...","['the', 'wii', 'version', 'was', 'released', '...",November 2006,False,1.692841e-232
3,2071,The_Legend_of_Zelda:_Twilight_Princess,When could GameCube owners purchase Twilight P...,"[when, could, gamecube, owners, purchase, twil...","['ADV', 'AUX', 'PROPN', 'NOUN', 'VERB', 'PROPN...","['WRB', 'MD', 'NNP', 'NNS', 'VB', 'NNP', 'NNP'...",NaN,The GameCube version was released worldwide in...,"[the, gamecube, version, was, released, worldw...",The Legend of Zelda: Twilight Princess (Japane...,"['the', 'legend', 'of', 'zelda', ':', 'twiligh...","['the', 'gamecube', 'version', 'was', 'release...",December 2006,False,0.000000e+00
4,2072,The_Legend_of_Zelda:_Twilight_Princess,What company developed Legend of Zelda: Twilig...,"[what, company, developed, legend, of, zelda, ...","['DET', 'NOUN', 'VERB', 'PROPN', 'ADP', 'PROPN...","['WDT', 'NN', 'VBD', 'NNP', 'IN', 'NNP', ':', ...",NaN,The Legend of Zelda: Twilight Princess (Japane...,"[the, legend, of, zelda, :, twilight, princess...",The Legend of Zelda: Twilight Princess (Japane...,"['the', 'legend', 'of', 'zelda', ':', 'twiligh...","['the', 'legend', 'of', 'zelda', ':', 'twiligh...",Nintendo,False,4.835285e-233


## Compare Question to target sentence as reference 

In [41]:
df["sentence_question_bleu"] =[sentence_bleu([question], sentence) for sentence, question in zip(df["question_token"], df["answer_sentence_token"])]
print("BLEU1: {}".format(corpus_bleu([[question] for question in df["question_token"]], df["answer_sentence_token"], weights=(1,0,0,0))))
print("BLEU2: {}".format(corpus_bleu([[question] for question in df["question_token"]], df["answer_sentence_token"], weights=(0,1,0,0))))
print("BLEU3: {}".format(corpus_bleu([[question] for question in df["question_token"]], df["answer_sentence_token"], weights=(0,0,1,0))))
print("BLEU4: {}".format(corpus_bleu([[question] for question in df["question_token"]], df["answer_sentence_token"], weights=(0,0,0,1))))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning:


The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning:


The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning:


The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()



BLEU1: 0.15836060768892582
BLEU2: 0.06648373423971089
BLEU3: 0.03510338239959503
BLEU4: 0.020003726289690354


In [42]:
import plotly.express as px
px.histogram(df, x="sentence_question_bleu")

# Compare to predicted Question

In [55]:
pred_file = "/content/gdrive/MyDrive/mt-qg-data/00_models/qg_attention/squad/dataset_squad-target_length_20-input_length_40-vocab_input_None-max_vocab_targ_None-epochs_15-units_600-batch_64-/dataset_squad-target_length_20-input_length_40-vocab_input_None-max_vocab_targ_None-epochs_15-units_600-batch_64-.txt"

In [56]:
pred_questions= []
with open(pred_file, "r") as f:
  for line in f:
    token = line.strip().split(" ")
    try:
      end_index = token.index("<end>")
    except ValueError:
      end_index = len(token)
    pred_questions.append(token[:end_index])

In [57]:
print(pred_questions[3])
print(df["question_token"][3])

['what', 'was', 'the', 'name', 'of', 'the', 'quickly', 'released', 'in', 'the', 'world', '?']
['when', 'could', 'gamecube', 'owners', 'purchase', 'twilight', 'princess', '?']


In [58]:
df["sentence_question_bleu"] =[sentence_bleu([question], sentence) for sentence, question in zip(df["question_token"], pred_questions)]
print("BLEU1: {}".format(corpus_bleu([[question] for question in df["question_token"]], pred_questions, weights=(1,0,0,0))))
print("BLEU2: {}".format(corpus_bleu([[question] for question in df["question_token"]], pred_questions, weights=(0,1,0,0))))
print("BLEU3: {}".format(corpus_bleu([[question] for question in df["question_token"]], pred_questions, weights=(0,0,1,0))))
print("BLEU4: {}".format(corpus_bleu([[question] for question in df["question_token"]], pred_questions, weights=(0,0,0,1))))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning:


The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning:


The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:516: UserWarning:


The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()



BLEU1: 0.2904992456708043
BLEU2: 0.04868244602939907
BLEU3: 0.015381753542981113
BLEU4: 0.00514905796279317


# First Token Confusion matrix

In [72]:
from sklearn.metrics import confusion_matrix
from collections import Counter
import plotly.graph_objects as go

In [85]:
def get_interrogative_phrase(question_token):
  return question_token[0]

def plot_confusion_matrix(y_true, y_pred):
    most_common = Counter(y_true).most_common(5)
    most_common_labels = [e[0] for e in most_common]
    my_confusion_matrix = confusion_matrix(y_true, y_pred, labels=most_common_labels, normalize="true")
    #my_confusion_matrix = my_confusion_matrix.astype(int)

    layout = {
        "title": "Confusion Matrix", 
        "xaxis": {"title": "Predicted value"}, 
        "yaxis": {"title": "Real value"}
    }

    fig = go.Figure(data=go.Heatmap(z=my_confusion_matrix,
                                    x=most_common_labels,
                                    y=most_common_labels,
                                    hoverongaps=False),
                    layout=layout)
    fig.show()

pred_wh = [get_interrogative_phrase(question) for question in pred_questions]
true_wh = [get_interrogative_phrase(question) for question in df["question_token"]]

In [86]:
plot_confusion_matrix(true_wh, pred_wh)